In [1]:
# Read in the csv file as a dataframe using pandas
import pandas as pd
df = pd.read_csv("quiz4_invest.csv", index_col='Project')

# read the parameters
Capital = {}
Sector = {}
Risk = {}
Duration = {}
ExpectedReturn = {}

for i in df.index:  # iterate over the row index
    Capital[i] = float(df['Capital ($M)'][i])
    Sector[i] = df['Sector'][i]
    Risk[i] = df['Risk'][i]
    Duration[i] = float(df['Duration'][i])
    ExpectedReturn[i] = float(df['Expected return ($M)'][i])
    
Projects = Capital.keys()
Budget = 100

In [2]:
from docplex.mp.model import Model
mdl = Model()

In [3]:

# Define decision variables
select = mdl.binary_var_dict(Projects, name='project')


In [4]:
# Objective function: maximize total expected return
mdl.maximize(mdl.sum(ExpectedReturn[i] * select[i] for i in Projects))



In [5]:
# Constraint: Total capital investment should not exceed $100M
mdl.add_constraint(mdl.sum(Capital[i] * select[i] for i in Projects) <= Budget, 'Total_Capital')



docplex.mp.LinearConstraint[Total_Capital](20project_Project 1+15project_Project 2+23project_Project 3+10project_Project 4+20project_Project 5+12project_Project 6+20project_Project 7+25project_Project 8+14project_Project 9+10project_Project 10,LE,100)

In [6]:
# Constraint: At most two projects in IT sector
mdl.add_constraint(mdl.sum(select[i] for i in Projects if Sector[i] == 'IT') <= 2, 'IT_Sector')



docplex.mp.LinearConstraint[IT_Sector](project_Project 1+project_Project 6+project_Project 10,LE,2)

In [7]:
# Constraint: At least one project in Healthcare sector
mdl.add_constraint(mdl.sum(select[i] for i in Projects if Sector[i] == 'Healthcare') >= 1, 'Healthcare_Sector')



docplex.mp.LinearConstraint[Healthcare_Sector](project_Project 2+project_Project 3,GE,1)

In [8]:
# Constraint: Number of low-risk projects must be at least the number of high-risk projects
mdl.add_constraint(mdl.sum(select[i] for i in Projects if Risk[i] == 'Low') >= 
                   mdl.sum(select[i] for i in Projects if Risk[i] == 'High'), 'Risk_Balance')




docplex.mp.LinearConstraint[Risk_Balance](project_Project 1+project_Project 8+project_Project 10,GE,project_Project 3+project_Project 6+project_Project 7)

In [9]:
# Constraint: At least three projects with duration of 4 years or shorter
mdl.add_constraint(mdl.sum(select[i] for i in Projects if Duration[i] <= 4) >= 3, 'Duration_Limit')



docplex.mp.LinearConstraint[Duration_Limit](project_Project 1+project_Project 2+project_Project 3+project_Project 4+project_Project 6+project_Project 9+project_Project 10,GE,3)

In [10]:
# solve
mdl.solve()
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0.703,status='integer optimal solution')

In [11]:
mdl.print_solution()

objective: 238.000
status: OPTIMAL_SOLUTION(2)
  "project_Project 1"=1
  "project_Project 2"=1
  "project_Project 4"=1
  "project_Project 5"=1
  "project_Project 7"=1
  "project_Project 9"=1


In [ ]:
# Question 3 Online

In [13]:
import pandas as pd
from docplex.mp.model import Model

# Load the data from the CSV file
df = pd.read_csv("quiz4_invest.csv", index_col='Project')

# Read the parameters
Capital = df['Capital ($M)'].to_dict()
Sector = df['Sector'].to_dict()
Risk = df['Risk'].to_dict()
Duration = df['Duration'].to_dict()
ExpectedReturn = df['Expected return ($M)'].to_dict()

Projects = list(Capital.keys())
Budget = 100  # Initial budget constraint remains, though it's not the primary constraint anymore

# Create the model
mdl = Model()

# Define decision variables
select = mdl.binary_var_dict(Projects, name='project')

# New variable for total capital used
total_capital = mdl.sum(Capital[i] * select[i] for i in Projects)

# Objective function: minimize total capital used
mdl.minimize(total_capital)

# Constraint: Expected return must be at least $200 million
mdl.add_constraint(mdl.sum(ExpectedReturn[i] * select[i] for i in Projects) >= 200, 'Expected_Return')

# Constraint: At most two projects in IT sector
mdl.add_constraint(mdl.sum(select[i] for i in Projects if Sector[i] == 'IT') <= 2, 'IT_Sector')

# Constraint: At least one project in Healthcare sector
mdl.add_constraint(mdl.sum(select[i] for i in Projects if Sector[i] == 'Healthcare') >= 1, 'Healthcare_Sector')

# Constraint: Number of low-risk projects must be at least the number of high-risk projects
mdl.add_constraint(mdl.sum(select[i] for i in Projects if Risk[i] == 'Low') >= 
                   mdl.sum(select[i] for i in Projects if Risk[i] == 'High'), 'Risk_Balance')

# Constraint: At least three projects with duration of 4 years or shorter
mdl.add_constraint(mdl.sum(select[i] for i in Projects if Duration[i] <= 4) >= 3, 'Duration_Limit')

# Solve the model
solution = mdl.solve()

# Output the solution
if solution:
    min_capital = total_capital.solution_value
    print("Minimum capital needed:", min_capital)
    print("Selected projects:")
    for p in Projects:
        if select[p].solution_value > 0.5:
            print(f"Project {p} selected")
else:
    print("No solution found")


Minimum capital needed: 79.0
Selected projects:
Project Project 2 selected
Project Project 5 selected
Project Project 7 selected
Project Project 9 selected
Project Project 10 selected
